# Exploration of `ExPeCoN`

Running analyses and simulations of `ExPeCoN`

___

    Author:  Carina Forster et al.
    Contact: forster@cbs.mpg.de
    Years:   2023

___

## Setup 

In [ ]:
# Import my expecon_ms package and other modules
from pathlib import Path

import numpy as np

from expecon_ms.configs import config, params, path_to

# Note that the jupyter kernel must be set to your (conda) environment in order to import your package

In [ ]:
# Set global vars & paths
pass

## Analyse 

In [ ]:
# Temporary dummy code
print("Current WD:", Path.cwd())

print("array:", np.array([1, 2, 3]))

print(f"{path_to.data.CACHE}/{params.weight_decay}/")

### Intermediate summary
This is a intermediate summary of the analysis.
We find this and that ...